# Robustness Noise 
In order to understand the results better from the testing so far, in particular the plots, we want to analyse how noise influences our computations.
This serves the overall goal of improving our method with noisy data.
Following questions arise:

How can we bound the finite difference method?

How stable is the SVD to noise?

We only consider noise which is normally distributed? Does this influence our result?

What is the relation between the highest and the lowest singular value? Do they grow/shrink in a similar way when confronted with noise?

## SVD Robustness Noise
There is a theorem and a corollary:

Theorem 2.18 (Mirsky). If $A,E \in K^{IxI}$ are two arbitrary matrices, then $\sqrt{\sum_{k=1}^{r}|\sigma_k(A+E) - \sigma_k(A)|} \leq \lVert E \rVert_F$

Corollary:
If $A,E \in K^{IxI}$ are two arbitrary matrices, then $\forall k \ |\sigma_k(A+E) - \sigma_k(A)| \leq \lVert E \rVert$

#### 1. What is the spectral and Frobenius norm of random noise matrices?

In [1]:
import numpy as np
from numpy.linalg import matrix_rank, svd
from test_data import experiment_data,add_noise
import pysindy as ps
import matplotlib.pyplot as plt

In [2]:
rows,cols = 10,3 #let rows be more than columns
min_value,max_value = -10,10
matrix = np.random.uniform(min_value, max_value, size=(rows, cols))
# Set col 2 equal to col 1
alpha = np.random.rand()
matrix[:, 1] = alpha*matrix[:,2]
print(matrix)

[[ 6.66562441 -0.76762407 -1.00446053]
 [ 8.34727277 -6.05521043 -7.92343562]
 [ 7.92623669  1.83994298  2.40762396]
 [-9.5471367   3.23763898  4.23655368]
 [ 8.03703868 -6.28227756 -8.22056019]
 [-7.34977118 -0.97541584 -1.27636268]
 [ 3.94674179  6.8205043   8.92484703]
 [-6.53621316 -2.74778225 -3.59556056]
 [-0.05338975 -5.22337025 -6.83494627]
 [ 3.07790248  0.36209434  0.47381197]]


In [3]:
print(matrix_rank(matrix),svd(matrix, compute_uv=False))
print(np.linalg.norm(matrix))
print(np.linalg.norm(matrix,ord=2))

2 [2.39961825e+01 1.86531719e+01 2.11036688e-15]
30.393380788364958
23.996182477906387


Add noise

In [4]:
#np.random.seed(12)
target_noise=1e-3
var = target_noise * np.sqrt(np.mean(np.square(matrix)))
noise = np.random.normal(0, var, size=matrix.shape)
matrix_noise = matrix + noise
print(f"Added Gaussian noise with variance {var}")

Added Gaussian noise with variance 0.005549046752210546


In [5]:
print("Noise Matrix")
#print(noise)
print(f"Matrix rank: {matrix_rank(noise)}, SVD: {svd(noise, compute_uv=False)}")
print(f"Frobenius norm:\t {np.linalg.norm(noise)}")
print(f"Spectral norm:\t {np.linalg.norm(noise,ord=2)}")

Noise Matrix
Matrix rank: 3, SVD: [0.02323391 0.01318877 0.00861189]
Frobenius norm:	 0.028069965626892308
Spectral norm:	 0.023233908649571117


Print SVD and matrix rank with additional noise

In [6]:
noise_levels = [0]+[10**(-10+i) for i in range(0,10)]
print("Noise level \t \t matrix rank \t svd \t \t \t \t \t \t \t Frobenius \t \t \t Spectral")
for target_noise in noise_levels:
    var = target_noise * np.sqrt(np.mean(np.square(matrix)))
    noise = np.random.normal(0, var, size=matrix.shape)
    matrix_noise = matrix + noise
    print(f" {target_noise}      \t \t {matrix_rank(matrix_noise)} \t \t {svd(matrix_noise, compute_uv=False)}     \t {np.linalg.norm(noise)}      \t {np.linalg.norm(noise,ord=2)}")

Noise level 	 	 matrix rank 	 svd 	 	 	 	 	 	 	 Frobenius 	 	 	 Spectral
 0      	 	 2 	 	 [2.39961825e+01 1.86531719e+01 2.11036688e-15]     	 0.0      	 0.0
 1e-10      	 	 3 	 	 [2.39961825e+01 1.86531719e+01 1.81789407e-09]     	 3.1287472580974682e-09      	 2.0729768842151004e-09
 1e-09      	 	 3 	 	 [2.39961825e+01 1.86531719e+01 1.82109851e-08]     	 2.7130037759048294e-08      	 1.9565171347580528e-08
 1e-08      	 	 3 	 	 [2.39961825e+01 1.86531719e+01 9.57945783e-08]     	 2.8046150381403314e-07      	 2.072792857946648e-07
 1e-07      	 	 3 	 	 [2.39961822e+01 1.86531719e+01 1.60265258e-06]     	 2.620569876147372e-06      	 1.943756058501899e-06
 1e-06      	 	 3 	 	 [2.39961910e+01 1.86531737e+01 1.32646444e-05]     	 2.973444612169634e-05      	 2.1971930105793356e-05
 1e-05      	 	 3 	 	 [2.39961844e+01 1.86532097e+01 1.51620628e-04]     	 0.00027272567005420144      	 0.00017252510215640522
 0.0001      	 	 3 	 	 [2.39960209e+01 1.86529741e+01 2.01561199e-03]     	 0

Although the Frobenius Norm might not be that high in relation to the highest singular value. In relation to the smallest singular value it can be quite significant. Thus, the changes in the smallest singular value can be more disturbing for our purpose.

#### 2. What is the ratio between the smallest and biggest singular value?

The idea is if $A \in \mathbb{R}^{nxm}$ for $m<n$ has full rank $\frac{\sigma_1}{\sigma_m} < \infty$, but if it does not have full rank $\frac{\sigma_1}{\sigma_m} = \infty$

In [7]:
#This can be negative and thus is not a correct bound on sv_max/sv_min
def calc_bound_ratio(sv_max,sv_min,noisematrix):
    #E=np.linalg.norm(noisematrix) #Spectral: order=2, Frobenius: default
    #B=1/sv_min+sv_max/sv_min**2
    E=np.linalg.norm(noisematrix,ord=2) #Spectral: order=2, Frobenius: default
    B=max(1/sv_min,sv_max/sv_min**2)
    return E*B

In [8]:
rows,cols = 10,3 #let rows be more than columns
min_value,max_value = -1/1000,1/1000
matrix = np.random.uniform(min_value, max_value, size=(rows, cols))
# Set col 2 equal to col 1
alpha = np.random.rand()
print(alpha)
matrix[:, 1] = alpha*matrix[:,2]
#matrix[:, 2] *=1/10000
#matrix[:, 1] *=1/10000
print(matrix)

0.17942386120429055
[[ 2.89858542e-04  1.57136307e-04  8.75782665e-04]
 [-5.28721303e-05  8.41100812e-06  4.68778682e-05]
 [ 9.10819416e-04 -2.47456658e-05 -1.37917363e-04]
 [ 9.53839408e-04  1.58538508e-05  8.83597684e-05]
 [ 9.42488968e-04 -1.26291842e-04 -7.03874284e-04]
 [ 8.81264927e-04  9.28196316e-05  5.17320444e-04]
 [ 4.65858680e-04  1.34582469e-04  7.50081220e-04]
 [-4.16504613e-04 -1.03292747e-04 -5.75691252e-04]
 [-6.60273042e-05 -1.47693442e-04 -8.23153851e-04]
 [ 8.38914954e-04  8.29977344e-05  4.62579134e-04]]


In [9]:
sv = svd(matrix, compute_uv=False)
sv_max = sv[0]
sv_min = sv[-1]
print(f"matrix shape: {matrix.shape}, rank: {matrix_rank(matrix)}" )
print(f"Singular values : {sv}" )
print(f"Ratio big/smalles singular value: {sv[0]/sv[-1]}")
print(f"Bound: {calc_bound_ratio(sv_max,sv_min,np.zeros(shape=matrix.shape))}")

matrix shape: (10, 3), rank: 2
Singular values : [2.28380860e-03 1.68117370e-03 7.81183361e-20]
Ratio big/smalles singular value: 2.923524386091933e+16
Bound: 0.0


In [11]:
noise_levels = [0]+[10**(-10+i) for i in range(0,10)]
print("Noise level \t matrix rank \t sv \t \t \t \t \t  ratio \t \t bound")
for target_noise in noise_levels:
    var = target_noise * np.sqrt(np.mean(np.square(matrix)))
    noise = np.random.normal(0, var, size=matrix.shape)
    matrix_noise = matrix+noise
    sv = svd(matrix_noise, compute_uv=False)
    print(f" {target_noise}\t \t {matrix_rank(matrix_noise)} \t \t {sv} \t  {sv[0]/sv[-1]}  \t {calc_bound_ratio(sv_max,sv_min,noise)}")

Noise level 	 matrix rank 	 sv 	 	 	 	 	  ratio 	 	 bound
 0	 	 2 	 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20] 	  2.923524386091933e+16  	 0.0
 1e-10	 	 3 	 	 [2.28380860e-03 1.68117370e-03 1.45214918e-13] 	  15727093597.293333  	 8.412736215642676e+22
 1e-09	 	 3 	 	 [2.28380860e-03 1.68117370e-03 1.03876909e-12] 	  2198571960.4897475  	 6.277132902507586e+23
 1e-08	 	 3 	 	 [2.28380860e-03 1.68117371e-03 1.77671631e-11] 	  128540982.15321995  	 8.416352523226794e+24
 1e-07	 	 3 	 	 [2.28380863e-03 1.68117370e-03 1.11087285e-10] 	  20558686.254647616  	 4.953413914875971e+25
 1e-06	 	 3 	 	 [2.28380825e-03 1.68117433e-03 1.40386291e-09] 	  1626802.9026847377  	 8.181370822911596e+26
 1e-05	 	 3 	 	 [2.28380928e-03 1.68116712e-03 1.27687334e-08] 	  178859.50020037135  	 5.46556662778185e+27
 0.0001	 	 3 	 	 [2.28375577e-03 1.68107048e-03 1.28072503e-07] 	  17831.741512301676  	 7.7868166436336415e+28
 0.001	 	 3 	 	 [2.28452049e-03 1.68174556e-03 1.25606774e-06] 	  1818.787647430

Since for matrix with not full rank the bound is very high, I want to test the deviation for the same noise level

In [20]:
trials=10
noise_level=1e-20
print(f"Noise level:  {noise_level}, Number trials: {trials}")
print("Trial \t matrix rank \t sv \t \t \t \t \t \t \t  ratio \t \t bound \t \t \t difference")
for trial in range(trials):
    var = noise_level * np.sqrt(np.mean(np.square(matrix)))
    noise = np.random.normal(0, var, size=matrix.shape)
    matrix_noise = matrix+noise
    sv = svd(matrix_noise, compute_uv=False)
    ratio=sv[0]/sv[-1]
    bound = calc_bound_ratio(sv_max,sv_min,noise)
    print(f" {trial}\t \t {matrix_rank(matrix_noise)} \t {sv}   \t  {ratio}   \t {bound} \t \t   {bound-np.abs(ratio-sv_max/sv_min)}")

Noise level:  1e-20, Number trials: 10
Trial 	 matrix rank 	 sv 	 	 	 	 	 	 	  ratio 	 	 bound 	 	 	 difference
 0	 	 2 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20]   	  2.923524386091933e+16   	 7461141177660.98 	 	   7461141177660.98
 1	 	 2 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20]   	  2.923524386091933e+16   	 10461507617493.658 	 	   10461507617493.658
 2	 	 2 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20]   	  2.923524386091933e+16   	 7149389622935.361 	 	   7149389622935.361
 3	 	 2 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20]   	  2.923524386091933e+16   	 7051346197861.786 	 	   7051346197861.786
 4	 	 2 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20]   	  2.923524386091933e+16   	 8460836717359.868 	 	   8460836717359.868
 5	 	 2 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20]   	  2.923524386091933e+16   	 7842799190085.322 	 	   7842799190085.322
 6	 	 2 	 [2.28380860e-03 1.68117370e-03 7.81183361e-20]   	  2.923524386091933e+16   	 6085147433042.157 	 

## Finite Difference Noise

We can find the coefficients use to calculate the finite differences for different orders in https://en.wikipedia.org/wiki/Finite_difference_coefficient. Calculator https://web.media.mit.edu/~crtaylor/calculator.html<br>
Assuming we have equispaced data for x,and we can bound the measurement error by $\epsilon > 0$ i.e. $||u-\tilde{u}||_{\infty}< \epsilon$ and the third derivative of u
is bounded by $M(t, x) > 0$ on each interval $[x − h, x + h]$. Then, we get tha
$$|u_x(t, x) − \tilde{u}_x(t, x)| ≤ \frac{\epsilon}{h} + \frac{h^2}{6}M(t,x)$$
is the 2nd order centered finite differences approximation of the derivative of $\tilde{u}$.

TODO: error bound for different orders, centered and left-off-centered/backward differences (even/odd order)?<br>
Question: do the same error bounds count at the boundary values of u?? For centered Differences no-> can lead to wron errorbound especially since exp has high values at boundary point

For even order: centered differenced, for uneven order backward differences:<br>
order = 1: <br>
Approximation: $\frac{u(x)-u(x-h)}{h}$,    Bound: $||u-\tilde{u}||_{\infty}<\frac{h}{2}||u'||_{\infty}+\frac{2\epsilon}{h}$ 
<br>
order = 2: <br>
Approximation: $\frac{u(x+\frac{h}{2})-u(x-\frac{h}{2})}{h}$,    Bound: $||u-\tilde{u}||_{\infty}<\frac{h^2}{3}||u^3||_{\infty}+\frac{2\epsilon}{h}$ <br>
(Here we use h/2 since for h the calculation did not work out -> makes more sense???)
order = 4: <br>
Approximation: see book or wiki,    Bound: $||u-\tilde{u}||_{\infty}<\frac{h^4}{30}||u^5||_{\infty}+\frac{9 \epsilon}{6h}$ <br>

In [ ]:
from test_data import create_data_2d
experiment_name = "linear_nonunique_1"
n_samples=200
T,X,t,x = create_data_2d(T_start=0, T_end=5, L_x_start=0,L_x_end=5, N_t=n_samples, N_x=n_samples)
a=np.random.randn()
#u = np.exp(X-a*T)
#ux = u
#uxx = u
#uxxx = u
#ut = -a*u

u = np.cos(X-a*T)
ux = -np.sin(X-a*T)
uxx = -np.cos(X-a*T)
uxxx = np.sin(X-a*T)
ut = a*np.sin(X-a*T)
dx=x[1]-x[0]

In [ ]:
ux_fd = ps.FiniteDifference(order=3,d=1, axis=0, drop_endpoints=False)._differentiate(u, dx)
u_flat, u_x_flat = u.flatten(), ux_fd.flatten()
g= np.concatenate([u_flat.reshape(len(u_flat),1), u_x_flat.reshape(len(u_flat),1)], axis=1)
print(g.shape)
print(f"Matrix rank = {matrix_rank(g)}, svd = {svd(g, compute_uv=False)}")

### Check difference between noisy derivative and correct derivative

General formula for central differences: $$ \text{sum over ceofficients} \frac{\epsilon}{h} + h^{order}\frac{(order/2)!^2}{(order+1)!}$$

For backward differences: $$ \text{sum over ceofficients}\frac{\epsilon}{h} + h^{order} 1/(order+1)$$

Here compute functions for computing lagrange coefficients and sum(for calculating measurment error)
$$L_{n,k}(x) =\prod_{i=0,i\neq k}^n \frac{x-x_i}{x_k-x_i}$$
$$  L'_{n,k}(x) = [\prod_{i=0,i\neq k}^n \frac{1}{x_k-x_i}] \sum_{j=0,j\neq k}^n \prod_{i=0,i\neq k,j}^n (x-x_i)
  = h^{-1}[\prod_{i=0,i\neq k}^n \frac{1}{k-i}] \sum_{j=0,j\neq k}^n \prod_{i=0,i\neq k,j}^n (l-i) $$

In [ ]:
"""
n: number of data points
k: number of lagrange coefficient
l: number of data point where L is evaluated at: x = x_l = x+ hl
"""
#Computes the coefficients of the derivitve of the kth Lagrangian Coefficients
#These are also the coefficients used for finite differences
def lagrange_coefficient_derivative(n,k,l):
    erg=0.0
    
    prod = 1.0
    for i in range(n+1):
        if i!=k:
            prod *= 1.0/(k-i)
            
    for j in range(n+1):
        if j!=k:
            tmp =1.0
            for i in range(n+1):
                if i!=k and i!=j:
                    tmp *=(l-i)
                    #print(f"l-k = {l}-{i}")
            erg+=tmp
    erg*=prod
    return erg
    
#Sums up the above coefficients
#needed for measurment error
def sum_lagrange_coefficient_derivative(n,l):
    erg=0.0
    for k in range(n+1):
        #print(f"L({n},{k},{l}) = {lagrange_coefficient_derivative(n,k,l)}")
        erg+= np.abs(lagrange_coefficient_derivative(n,k,l))
    return erg

In [ ]:
# Approximation error central differences: reduces with higher order n
def appr_error_central_diff(n):
    assert(n%2==0) #Check if n is even
    bound = (np.math.factorial(int(n/2))**2)/np.math.factorial(n+1)
    return bound
    
# Approximation error backward differences: reduces with higher order n
def appr_error_backward_diff(n):
    return 1/(n+1)

# Measurement error central differences
def meas_error_central_diff(n):
    assert(n%2==0) #Check if n is even
    erg=sum_lagrange_coefficient_derivative(n,n/2)
    return erg
    
# Measurement error backward differences
def meas_error_backward_diff(n):
    erg=sum_lagrange_coefficient_derivative(n,0)
    return erg

In [ ]:
for i in range(1,20,2):
    print(f"Order: {i}, Error: {meas_error_backward_diff(i)}")

In [ ]:
#eps: measurement error+roundof error (Does it make sense to add machine precision?)
#h: dx for equispaced data
#M: bound on nth derivative depending on the order
def error_bound(eps,h,M,order=2):
    eps+=np.finfo(float).eps # add machine precisoin
    if order%2==0: #even order
        C_app=appr_error_central_diff(order)
        C_meas=meas_error_central_diff(order)
    else: # odd order
        C_app=appr_error_backward_diff(order)
        C_meas=meas_error_backward_diff(order)
        
    return C_meas*eps/h + (h**order)*M*C_app
            
def infinity_norm(x):
    return np.max(np.abs(x))

In [ ]:
target_noise=1e-9
var = target_noise * np.sqrt(np.mean(np.square(u)))
noise = np.random.normal(0, var, size=u.shape)
u_noise = u + noise
order=2
ux_noise = ps.FiniteDifference(order=order,d=1, axis=0, drop_endpoints=False)._differentiate(u_noise, dx)

In [ ]:
infinity_norm(ux-ux_noise)

In [ ]:
eps=infinity_norm(u_noise-u)
M=infinity_norm(uxxx)
h=dx
print(f"eps: {eps}, M: {M}, h: {h}")

In [ ]:
error_bound(eps,h,M,order=order)

Error bound smaller than difference for oder =^1? -> wrong formula
three poiint midpoint ?/ numerical mistakes in error bound calcuation?<br>
What about the boundary values for central diff??? -> I think they are the reason the bound fails for some noise levels since we can not use central differnce at the boundary values!

In [ ]:
M=infinity_norm(uxxx)
noise_levels = [0]+[10**(-10+i) for i in range(0,10)]
print("Noise level \t  eps \t \t $max|ux-ux_noise|$ \t bound")
plt.title("Derivative Cutout")
order=2
for target_noise in noise_levels:
    var = target_noise * np.sqrt(np.mean(np.square(u)))
    noise = np.random.normal(0, var, size=u.shape)
    u_noise = u + noise
    ux_noise = ps.FiniteDifference(order=order,d=1, axis=0, drop_endpoints=False)._differentiate(u_noise, dx)
    plt.plot(ux_noise[0,:],label=str(target_noise))
    eps = infinity_norm(u_noise-u)
    bound = error_bound(eps,dx,M,order)
    diff = infinity_norm(ux-ux_noise)
    print(f"{target_noise} \t \t  {eps:.4f} \t {diff:.4f} \t \t {bound:.4f}")
plt.legend()
plt.show()

In [ ]:
M=infinity_norm(uxxx)
number_noise=10 #<=10
noise_levels = [0]+[10**(-10+i) for i in range(0,number_noise-1)]
orders=range(1,10,1)
print("Order \t Noise level \t  eps \t \t|ux-ux_noise| \t bound")
diff_list = []
bound_list = []
for order in orders:
    print(f"{order}")
    avg_diff=0
    avg_bound=0
    for target_noise in noise_levels:
        var = target_noise * np.sqrt(np.mean(np.square(u)))
        noise = np.random.normal(0, var, size=u.shape)
        u_noise = u + noise
        ux_noise = ps.FiniteDifference(order=order,d=1, axis=0, drop_endpoints=False)._differentiate(u_noise, dx)
        eps = infinity_norm(u_noise-u)
        bound = error_bound(eps,dx,M,order)
        diff = infinity_norm(ux-ux_noise)
        avg_bound+=bound
        avg_diff+=diff
        print(f"\t {target_noise}\t \t  {eps:.3f} \t {diff:.4f} \t {bound:.4f}")
    avg_bound/=number_noise
    avg_diff/=number_noise
    bound_list.append(avg_bound)
    diff_list.append(avg_diff)

In [ ]:
plt.title("Average bound and  |ux-ux_noise|")
plt.xlabel("Order")
plt.plot(orders,bound_list, marker='o',label='bounds')
plt.plot(orders,diff_list,marker='x',label='|ux-ux_noise|')
plt.legend()

#### Without boundary values

In [ ]:
M=infinity_norm(uxxx)
number_noise=8 #<=10
noise_levels = [0]+[10**(-10+i) for i in range(0,number_noise-1)]
orders=range(1,10,1)
print("Order \t Noise level \t  eps \t \t|ux-ux_noise| \t bound")
diff_list = []
bound_list = []
for order in orders:
    print(f"{order}")
    avg_diff=0
    avg_bound=0
    for target_noise in noise_levels:
        var = target_noise * np.sqrt(np.mean(np.square(u)))
        noise = np.random.normal(0, var, size=u.shape)
        u_noise = u + noise
        ux_noise = ps.FiniteDifference(order=order,d=1, axis=0, drop_endpoints=True)._differentiate(u_noise, dx)
        row_mask = ~np.isnan(ux_noise).all(axis=1)
        col_mask = ~np.isnan(ux_noise).all(axis=0)
        eps = infinity_norm(u_noise[row_mask][:, col_mask]-u[row_mask][:, col_mask])
        bound = error_bound(eps,dx,M,order)
        diff = infinity_norm(ux[row_mask][:, col_mask]-ux_noise[row_mask][:, col_mask])
        avg_bound+=bound
        avg_diff+=diff
        print(f"\t {target_noise}\t \t  {eps:.3f} \t {diff:.4f} \t {bound:.4f}")
    avg_bound/=number_noise
    avg_diff/=number_noise
    bound_list.append(avg_bound)
    diff_list.append(avg_diff)

In [ ]:
plt.title("Average bound and  |ux-ux_noise|")
plt.xlabel("Order")
plt.plot(orders,bound_list, marker='o',label='bounds')
plt.plot(orders,diff_list,marker='x',label='|ux-ux_noise|')
plt.legend()